# Daten importieren

In [1]:
#from dectrees_interactive_tools import ClassificationTree as ct
from PyTree import ClassificationTree as ct

import pathlib
import os
path = str(pathlib.Path().absolute()) + "/Daten/Training" 
files = os.listdir(path)


import pandas as pd
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual, Layout, HBox
import ipywidgets as widgets

import plotly as py
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()

In [2]:
def read_csv(Dateiname):
    df_train = pd.read_csv('Daten/Training/'+Dateiname, sep = ';')
    df_test = pd.read_csv('Daten/Test/'+Dateiname, sep = ';')
    print('Trainingsdaten')
    display(df_train)
    print('Testdaten')
    display(df_test)
    return df_train, df_test

data_widget=interactive(read_csv, Dateiname = files)
#data_widget.result = pd.DataFrame(columns=['Importiere Datensatz'])
data_widget

interactive(children=(Dropdown(description='Dateiname', options=('LebensmittelDaten.csv', 'jim_numerisch.csv',…

# Entscheidungsbaum trainieren

In [3]:
def grow_tree(Zielvariable, Kriterium, Max_Tiefe):
    
    if Kriterium == 'Fehlklassifikationsrate':
        Krit = 'misclassification_error'
    else:
        Krit = 'entropy'

    tree = ct.DecisionTree(data = data_widget.result[0], # das sind die Trainingsdaten
                           target = Zielvariable, 
                           crit = Krit)
    
    tree.grow_tree(max_depth = Max_Tiefe-1, crit = Krit)
    tree.print_tree()
    display(tree.tree_graph)

    print('Rate der korrekten Klassifikationen: \n')
    print('Trainingsdaten',round((tree.prediction_accuracy(data_widget.result[0]))*100,1),'% \n')
    
    print('Testdaten',round(tree.prediction_accuracy(data_widget.result[1])*100,1),'%') 


    return tree
    
tree_widget = interactive(grow_tree,{'manual': True, 'manual_name': 'Erstelle Tree'},
                          Zielvariable = data_widget.result[0].columns.sort_values(),
                          Kriterium=['Fehlklassifikationsrate','Entropie'],
                          Max_Tiefe=(1,20,1) ) 
tree_widget


interactive(children=(Dropdown(description='Zielvariable', options=('Eiweiß', 'Energie', 'Fett', 'Kohlenhydrat…

## Verlauf des Trainingsprozesses Visualisieren

In [4]:
def analyse_training(Kriterium):
    
    if Kriterium == 'Tiefe':
        tree_widget.result.evaluation_depth(data_widget.result[0],data_widget.result[1]).iplot(xTitle='Tiefe des Baums', yTitle='Rate korrekter Klassifikationen')
    elif Kriterium == 'Knoten':
        tree_widget.result.evaluation_node_count(data_widget.result[0],data_widget.result[1]).iplot(xTitle='Anzahl der Knoten', yTitle='Rate korrekter Klassifikationen')
                        
    
analyse_tree_widget = interactive(analyse_training,{'manual': True, 'manual_name': 'Verlauf analysieren'},Kriterium=['Tiefe','Knoten'])
analyse_tree_widget


interactive(children=(Dropdown(description='Kriterium', options=('Tiefe', 'Knoten'), value='Tiefe'), Button(de…

## Entscheidungsbaum  manuell beschneiden (Pruning)

In [5]:
def prune_trial(Knoten_nr, permanent):
    
    if permanent:
        tree_widget.result.prune_node(int(Knoten_nr))
        tree_widget.result.print_tree()
        display(tree_widget.result.tree_graph)
        
        print('Rate der korrekten Klassifizierungen: \n')
        print('Trainingsdaten',round((tree_widget.result.prediction_accuracy(data_widget.result[0]))*100,5),'% \n')
        print('Testdaten',round(tree_widget.result.prediction_accuracy(data_widget.result[1])*100,5),'%')      
    else:
        current_tree = tree_widget.result.copy()
        current_tree.prune_node(int(Knoten_nr))
        current_tree.print_tree()
        display(current_tree.tree_graph)
    
        print('Rate der korrekten Klassifikationen: \n')
        print('Trainingsdaten',round((current_tree.prediction_accuracy(data_widget.result[0]))*100,5),'% \n')
        print('Testdaten',round(current_tree.prediction_accuracy(data_widget.result[1])*100,5),'%')      
    
    
    return current_tree
    
prune_widget_trial = interactive(prune_trial,{'manual': True, 'manual_name': 'Knoten abschneiden'}, Knoten_nr = '14',  permanent = False)
prune_widget_trial

interactive(children=(Text(value='14', description='Knoten_nr'), Checkbox(value=False, description='permanent'…

## Entscheidungsbaum Ausgeben

In [6]:
def output_tree():       
    tree_widget.result.tree_graph.view()
    
output_tree_widget = interactive(output_tree,{'manual': True, 'manual_name': 'Tree ausgeben'})
output_tree_widget


interactive(children=(Button(description='Tree ausgeben', style=ButtonStyle()), Output()), _dom_classes=('widg…